# Average top public notebooks

Weighted mean in cartesian coordinates at equal altitude.

**All credit to the authors of those notebooks; please look at their work:**
* [GSDC phones mean prediction](https://www.kaggle.com/t88take/gsdc-phones-mean-prediction)
* [device EDA & Interpolate by removing device[en,ja]](https://www.kaggle.com/columbia2131/device-eda-interpolate-by-removing-device-en-ja)
* [GSDC: Position shift](https://www.kaggle.com/wrrosa/gsdc-position-shift)

In [1]:
import os
import numpy
import pandas as pd
import wgs_ecef

In [2]:
!ls ../input/

device-eda-interpolate-by-removing-device-en-ja  gsdc-phones-mean-prediction
google-smartphone-decimeter-challenge		 gsdc-position-shift


In [3]:
gsdc_position_shift = pd.read_csv('../input/gsdc-position-shift/submission.csv')
gsdc_phones_mean_prediction = pd.read_csv('../input/gsdc-phones-mean-prediction/submission.csv')
google_smartphone_decimeter_challenge = pd.read_csv('../input/device-eda-interpolate-by-removing-device-en-ja/submission.csv')

In [4]:
import numpy as np

blend_weights_lat = {
    "gsdc-phones-mean-prediction": (7.8 + 7.625) / 2 + 0.0625 + 0.0625/2 - 0.0625/4 + 0.0625/8,
    "device-eda-interpolate-by-removing-device-en-ja":  (2.25 + 2.5) / 2,
    "gsdc-position-shift":  0.75 - 0.125,
}


blend_weights_long = {
    "gsdc-phones-mean-prediction": (7.5 + 7.625) / 2 + 0.0625  + 0.0625/2 - 0.0625/4 + 0.0625/8,
    "device-eda-interpolate-by-removing-device-en-ja":(2.25 + 2.5) / 2,
    "gsdc-position-shift": 0.75 - 0.125,
}

lat_norm = sum(blend_weights_lat.values())
long_norm = sum(blend_weights_long.values())

blend = gsdc_phones_mean_prediction
# blend['millisSinceGpsEpoch'] = (
#     gsdc_position_shift['millisSinceGpsEpoch'] * blend_weights["gsdc-position-shift"]/norm +
#     gsdc_phones_mean_prediction['millisSinceGpsEpoch'] * blend_weights["gsdc-phones-mean-prediction"]/norm +
#     google_smartphone_decimeter_challenge['millisSinceGpsEpoch'] * blend_weights["device-eda-interpolate-by-removing-device-en-ja"]/norm
# )


blend['latDeg'] = (
    gsdc_position_shift['latDeg'] **2 * blend_weights_lat["gsdc-position-shift"]/lat_norm +
    gsdc_phones_mean_prediction['latDeg'] **2 * blend_weights_lat["gsdc-phones-mean-prediction"]/lat_norm +
    google_smartphone_decimeter_challenge['latDeg']**2  * blend_weights_lat["device-eda-interpolate-by-removing-device-en-ja"]/lat_norm 
)**0.5

blend['lngDeg'] = (
    gsdc_position_shift['lngDeg'] * blend_weights_long["gsdc-position-shift"]/long_norm +
    gsdc_phones_mean_prediction['lngDeg'] * blend_weights_long["gsdc-phones-mean-prediction"]/long_norm +
    google_smartphone_decimeter_challenge['lngDeg'] * blend_weights_long["device-eda-interpolate-by-removing-device-en-ja"]/long_norm 
)



In [5]:
(4**2)**0.5

4.0

In [6]:
blend['millisSinceGpsEpoch'] = blend['millisSinceGpsEpoch'].astype(np.int64)

In [7]:
blend.to_csv("submission.csv", index=False)